In [1]:
!git clone https://github.com/saadarshad102/Chatbot-Seq2Seq.git

Cloning into 'Chatbot-Seq2Seq'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 16 (delta 4), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [1]:
import numpy as np
import pandas as pd
from keras import layers,activations,models,preprocessing,utils
import tensorflow as tf

print( tf.VERSION )

Using TensorFlow backend.


1.15.0


In [0]:
datapath='/content/Chatbot-Seq2Seq/clean_conversation.txt'

In [0]:
inp=[]
target=[]

with open(datapath,'r', encoding='utf-8') as f:
    line=f.read().split('\n')
    
for i in line[:min(600,len(line)-1)]:
    inpt=i.split('\t')[0]
    targett=i.split('\t')[1]
    inp.append(inpt)
    target.append(targett)
    

In [0]:
zipped=list(zip(inp,target))

In [0]:
data=pd.DataFrame(zipped, columns=['input','output'])

In [6]:
data.head()

,input,output
0,What are your interests,I am interested in all kinds of things. We can...
1,What are your favorite subjects,"My favorite subjects include robotics, compute..."
2,What are your interests,"I am interested in a wide variety of topics, a..."
3,What is your number,I don't have any number
4,What is your number,23 skiddoo!


In [0]:
input_lines=list()
for i in data.input:
    input_lines.append(i)
    
    
tokenizer=preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(input_lines)
tokenized_input=tokenizer.texts_to_sequences(input_lines)

In [8]:
length_list=[]
for i in tokenized_input:
    length_list.append(len(i))
    
max_input_length=np.array(length_list).max()

print("Max length {}".format(max_input_length))
  

Max length 22


In [9]:
padded_input_lines=preprocessing.sequence.pad_sequences(tokenized_input, maxlen=max_input_length,padding='post')
encoder_input_data=np.array(padded_input_lines)
print("Encoder inputs {}".format(encoder_input_data.shape))

Encoder inputs (566, 22)


In [10]:
input_word_dict=tokenizer.word_index
num_input_tokens=len(input_word_dict)+1
print("Number of input tokens {}".format(num_input_tokens))

Number of input tokens 518


In [11]:
output_lines=list()
for i in data.output:
    output_lines.append('<start>'+ i + '<end>')
    
    
tokenizer=preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(output_lines)
tokenized_output_lines=tokenizer.texts_to_sequences(output_lines)



length_list=list()
for i in tokenized_output_lines:
    length_list.append(len(i))
max_output_length=np.array(length_list).max()
print( 'Output max length is {}'.format( max_output_length ))

padded_output_lines=preprocessing.sequence.pad_sequences(tokenized_output_lines,maxlen=max_output_length,padding='post')
decoder_input_data=np.array(padded_output_lines)
print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))

output_word_dict=tokenizer.word_index
num_output_tokens=len(output_word_dict)+1
print( 'Number of Output tokens = {}'.format( num_output_tokens))   


Output max length is 74
Decoder input data shape -> (566, 74)
Number of Output tokens = 1692


In [12]:
decoder_target_data=list()
for i in tokenized_output_lines:
    decoder_target_data.append(i[1:])
    
padding_output_lines=preprocessing.sequence.pad_sequences(decoder_target_data,maxlen=max_output_length,padding='post')
onehot_output_lines=utils.to_categorical(padding_output_lines,num_output_tokens)
decoder_target_data=np.array(onehot_output_lines)
print("Decoder target shape {}".format(decoder_target_data.shape))

Decoder target shape (566, 74, 1692)


In [14]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_input_tokens, 256 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 256 , return_state=True , recurrent_dropout=0.2 , dropout=0.2 )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_output_tokens, 256 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 256 , return_state=True , return_sequences=True , recurrent_dropout=0.2 , dropout=0.2)
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_output_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy')

model.summary()


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    132608      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embed

In [15]:
model.fit([encoder_input_data, decoder_input_data],decoder_target_data,batch_size=124, epochs=250)
model.save('model.h5')

Train on 566 samples
Epoch 1/250
566/566 [==============================] - 17s 30ms/sample - loss: 1.4407
Epoch 2/250
566/566 [==============================] - 14s 25ms/sample - loss: 1.4347
Epoch 3/250
566/566 [==============================] - 14s 25ms/sample - loss: 1.3810
Epoch 4/250
566/566 [==============================] - 14s 25ms/sample - loss: 1.2217
Epoch 5/250
566/566 [==============================] - 14s 25ms/sample - loss: 1.1489
Epoch 6/250
566/566 [==============================] - 14s 25ms/sample - loss: 1.1188
Epoch 7/250
566/566 [==============================] - 14s 25ms/sample - loss: 1.1119
Epoch 8/250
566/566 [==============================] - 14s 25ms/sample - loss: 1.1041
Epoch 9/250
566/566 [==============================] - 14s 25ms/sample - loss: 1.0976
Epoch 10/250
566/566 [==============================] - 14s 25ms/sample - loss: 1.0933
Epoch 11/250
566/566 [==============================] - 14s 25ms/sample - loss: 1.0867
Epoch 12/250
566/566 [=========

In [0]:
def inference_model():
    
      encoder_model=tf.keras.models.Model(encoder_inputs, encoder_states)
      
      decoder_state_h=tf.keras.layers.Input(shape=(256,))
      decoder_state_c=tf.keras.layers.Input(shape=(256,))
      
      
      decoder_state_inputs=[decoder_state_h, decoder_state_c]
      decoder_outputs,state_h,state_c=decoder_lstm(decoder_embedding,initial_state=decoder_state_inputs)
      decoder_states = [ state_h , state_c ]
      decoder_outputs=decoder_dense(decoder_outputs)
      decoder_model=tf.keras.models.Model([decoder_inputs]+ decoder_state_inputs,[decoder_outputs]+decoder_states)
      return encoder_model,decoder_model
      

In [0]:
def str_to_tokens(sentence : str):
    
      words=sentence.lower().split()
      token_list=list()
      for word in words:
          token_list.append(input_word_dict[word])
      
      return preprocessing.sequence.pad_sequences([token_list], maxlen=max_input_length,padding='post')

In [0]:
enc_model , dec_model = inference_model()

enc_model.save( 'enc_model.h5' ) 
dec_model.save( 'dec_model.h5' ) 
model.save( 'model.h5' ) 

for epoch in range( encoder_input_data.shape[0] ):
    states_values = enc_model.predict( str_to_tokens( input( 'User: ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = output_word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in output_word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( "Bot:" +decoded_translation.replace(' end', '') )
    print()

User: hi
Bot: hello

User: what is ai
Bot: artificial intelligence is the branch of engineering and science devoted to constructing machines that think

User: who are you
Bot: i am just an artificial intelligence

